*Authors:* 

In [ ]:
%%html
<style>
.jp-RenderedHTMLCommon pre, .jp-RenderedHTMLCommon pre > code {
    background-color: white;
    color: black;
    font-weight: 600;
}
.jp-RenderedHTMLCommon pre {
    padding: 10px;
    border: solid black;
}
</style>

# Lesson 11: Scripts and Software Engineering

*Goals*: Executable scripts, styleguides, and hints for Python development

Often software development and coding are not a single-person effort but involve collaborating with several people to achieve an overarching goal. Coding in physics is no exception to this. 

This means that, at one point or another, someone will need to read and understand the code that you wrote. To make this as easy and fast as possible, it is good practice to follow a broad series of styles and conventions. 




# Technical Debt
The following part may be interpreted as a little "philosophical" but its actually important to understand the concept of technical debt and its consequences for the development of projects.
Every time you do a coding decision you your project may accumulate technical debt. 
You need to understand that your project is not a one-time thing, but something that will evolve over time.
The project will be maintained, updated, and improved overtime and you need to make sure that the code is easy to understand and modify.

#### Technical debt covers:
When people talk about technical debt the following topics are typically covered:
- lack of documentation
- lack of structure
- badly written code
- code that breaks in special cases
- unexpected behavior (bugs)

#### Why does technical debt accumulate?
There are several reasons why technical debt accumulates:
- Haste: Pressure to finish quickly teases programmers to cut corners. It is okay to say: "I can clean this up later.", but it is not okay to never do this. Slowing down your pace of programming under pressure takes courage.

- Misunderstanding the problem: Many people learn more about circumstances of the problem they want to solve, while writing code about it. It is almost inevitably that some of the progress you made in between turn out to be wrong. Every time you add a new code to correct your wrong assumptions, they will lay a burden on the original design, unless you clean up properly.

- Lack of experience: An unexperienced programmer thinks that programming means writing code. Lack of experience often results in code that is unnecessary long or complicated. And even experienced programmers can create problematic code, simply because they came from another programming language or have a different style.

- Limitations of your used programming language: Python checks for SyntaxErrors and the most obvious exceptions at runtime. Unfortunately, Python does not notice much more. This is something most of you already experienced, when handing in a solution. To prevent this kind of behavior an IDE (Integrated Development Environment) can be used. More on this later. 

- Changes in the environment: Even if your program is written perfectly, it will slowly deteriorate, since its dependencies will change. After sometime your program will depend on very old software with deprecated methods. To stay up to date technically, the code needs to adapt and it would be good to be able to do this without much effort.

After learning the concepts of technical debt, you should be able to understand why it is important to write clean code and now we look into how to improve our coding style.

# Maintainability
Jupyter Notebooks are great for prototyping and sharing your work, since your code is directly visible and can be executed.
However, they are not the best tool for writing maintainable code, simply because they are not designed to be reuseable.
This is were python scripts come in.

## Python scripts
We have already seen how to create *modules*, which are `.py` files that contain Python code and can be imported. 
The "normal" way of writing programs is as *scripts*, that contain code that is supposed to be executed, e.g. from a terminal.
To not leave the notebook here, we will again use the `writefile` magic command. In reality you would probably do it in an editor.

In [ ]:
# create a directory for our scripts
%mkdir -p myscripts

In [ ]:
%%writefile myscripts/hello_world.py

print("Hello World")

The script is now located in the directory of your notebook. 

You can execute the script by running the `python3` executable (`python3` is used on most systems for Python version 3, while the `python` executable can also point to `python2`) and specifying the path to your script as the argument. 
We use the relative path from here:
```
$ python3 myscripts/hello_world.py 
Hello World
```

This script of course does not do much, but it is a start and what is more important, it is a file that can be shared and reused.

### Command line arguments
Scripts can take arguments from the command line, making them much more versatile to use. 
We'll first define a function that takes a parameter and that we'll call within the script:

In [ ]:
%%writefile myscripts/counter.py
import time

def count_to(n):
    for i in range(n):
        print(i + 1, end="\r")
        time.sleep(1)  # wait for a second to pretend we are doing something useful
    print("Counting finished :-)")
count_to(10)

Run the script and see what happens. If you chose a number that was too large and the script takes too long, you can interrupt it by pressing `Ctrl`+`c` (`Strg`+`c`), this is the universal way to interrupt a running program in the terminal.
```
$ python3 myscripts/counter.py
```
Now we will make the number we want to count to a *command line argument*:

In [ ]:
%%writefile myscripts/counter.py
import sys
import time

def count_to(n):
    for i in range(n):
        print(i + 1, end="\r")
        time.sleep(1)  # wait for a second to pretend we are doing something useful
    print("Counting finished :-)")

# try this first to see what is stored here; call it with different numbers of parameters:
print(type(sys.argv))
print(sys.argv)

```
$ python3 myscripts/counter.py
$ python3 myscripts/counter.py 1
$ python3 myscripts/counter.py 1 2
$ python3 myscripts/counter.py 1 2 a
$ python3 myscripts/counter.py 1 2 a dieter
```

It is a list containing the the script name followed by all command line arguments. We'll now make sure that exactly 1 argument is specified and use that as the argument for our function call.

In [ ]:
%%writefile myscripts/counter.py
import sys
import time

def count_to(n):
    for i in range(n):
        print(i + 1, end="\r")
        time.sleep(1)  # wait for a second to pretend we are doing something useful
    print("Counting finished :-)")

if len(sys.argv) != 2:
    print("One parameter is expected (the number to count to)! Exiting.")
else:
    # the arguments are strings
    count_to(int(sys.argv[1]))

Now run
```
$ python3 myscripts/counter.py
$ python3 myscripts/counter.py 5
$ python3 myscripts/counter.py 5 3
$ python3 myscripts/counter.py noint
```

### Argparse handling more than 1 argument
For real scripts you will probably need more than one parameter and you want to utilize named parameters, boolean settings (called flags), and of course default parameters. 

For this kind of problem it is highly recommended to you argument parser. These are interfaces to handle given arguments, the built-in parser for python is [argparse](https://docs.python.org/3/library/argparse.html). In the following you will see our counter scripts with argparse.



In [ ]:
%%writefile ./counter_with_parser.py
import time
import argparse


def count_to(n, print_numbers=False, ending_symbol="\r"):
    for i in range(n):
        if print_numbers:
            print(i + 1, end=ending_symbol)
        time.sleep(1)  # wait for a second to pretend we are doing something useful
    print("Counting finished :-)")


if __name__ == "__main__":
    # create an ArgumentParser object and add arguments
    parser = argparse.ArgumentParser(description="Count to a number")

    # you can set the argument with --number or -n in the command line
    # type: it is interpreted as an integer (default is str)
    # dest: the attribute name in the Namespace object
    # required is set to True: The argument must be provided (the same as our if in the previous script)
    # help: is the text users see when calling the script with --help
    # choices: the user can only choose from the provided list, everything else is permitted
    parser.add_argument(
        "--number",
        "-n",
        dest="n",
        type=int,
        choices=range(1, 100),
        help="The number to count to",
        required=True,
    )
    # action="store_true", if the argument is provided, the attribute will be set to True
    # if not provided, it will be False,
    # default is "store", meaning the value is saved as variable
    parser.add_argument(
        "--print_numbers",
        "-p",
        dest="print_numbers",
        action="store_true",
        required=True,
        help="Print the numbers while counting",
    )

    # args will be a Namespace object with the argument as attributes
    args = parser.parse_args()

    # run the function with the provided argument
    count_to(n=args.n, print_numbers=args.print_numbers)

Now run the script using your arguments with -n and -p, or by using the actual name.
```
$ python3 myscripts/counter.py -n 5 -p

# this will not work, since it is outside of our choices
$ python3 myscripts/counter.py -n 101 -p

# will also not work since -e is not a valid argument
$ python3 myscripts/counter.py --number 101 --print -e
```
This way you can provide a lot of different arguments to your script and make it very versatile to use over the command line interface.

### Mixing scripts and modules

We already know that we can import other Python files as modules (from a notebook, from a script, or from another module). E.g., if we would like to use the function we defined in the Python file, we can also import it!

In [ ]:
import myscripts.counter

The code inside the script gets executed, which is not what we wanted!

We can "guard" the execution to restrict it to be run only when invoked as a script. There is the special `__name__` variable that can be used:

In [ ]:
%%writefile myscripts/counter.py
import sys
import time

def count_to(n):
    for i in range(n):
        print(i + 1, end="\r")
        time.sleep(1)  # wait for a second to pretend we are doing something useful
    print("Counting finished :-)")

if __name__ == "__main__":
    if len(sys.argv) != 2:
        print("One parameter is expected (the number to count to)! Exiting.")
    else:
        # the arguments are strings
        count_to(int(sys.argv[1]))

Another reason why scripts are useful is that they start with a clean slate. This means that all variables are undefined and all imports are done again. This is not the case in a notebook, where you can run cells in any order and variables are kept in memory. 
To reload our module in a notebook, we can use the `reload` function from the `importlib` module:

In [ ]:
import importlib
importlib.reload(myscripts.counter)

Now the print is no longer executed at import time, but we can use the function:

In [ ]:
myscripts.counter.count_to(3)

It is very [idiomatic](https://docs.python.org/3/library/__main__.html#idiomatic-usage) to define a main function that is called within the `if` block, e.g., to avoid having global variables.

It can (and should) have an integer return value that is passed to `sys.exit()`.
An exit code of 0 signals to the operating system that the program ran successfully.
[Exit codes other than 0](https://tldp.org/LDP/abs/html/exitcodes.html) mean that there was some kind of error.

`sys.exit()` immediately stops the execution of the Python program. If you do not explicitly call, 0 is returned as exit code.

In [ ]:
%%writefile myscripts/counter.py
import sys
import time

def count_to(n):
    for i in range(n):
        print(i + 1, end="\r")
        time.sleep(1)  # wait for a second to pretend we are doing something useful
    print("Counting finished :-)")

def main():
    if len(sys.argv) != 2:
        print("One parameter is expected (the number to count to)! Exiting.")
        return 1  # no success
    # the arguments are strings
    count_to(int(sys.argv[1]))
    return 0  # success

if __name__ == "__main__":
    sys.exit(main())

You can check the exit code of the previous execution using the special `$?` environment variable. Note that also in case of a standard Python error (last example) the exit code is non-zero:
```
$ python3 myscripts/counter.py
$ echo $?
$ python3 myscripts/counter.py 1
$ echo $?
$ python3 myscripts/counter.py asdf
$ echo $?
```

For the rest of this lesson we will return mainly to our notebook-way of doing things. Some of the following recommendations will make only sense in the case of modules and scripts.

### Use Functions

Writing code in one large block quickly leads to code that can be hard to understand. One way around this is to segment your code sections into smaller functions, as was introduced in lesson 06. Better readability presents an additional benefit to the avoidance of repetition discussed in lesson 06.

In [ ]:
## Say we want to compute the Fibonacci sequence. We can directly implement this as such:

n = 10

if n == 0:
    fib_n = 0
elif n == 1:
    fib_n = 1
else:
    fib_n_minus_2 = 0
    fib_n_minus_1 = 1
    for i in range(2, n + 1):
        fib_n = fib_n_minus_1 + fib_n_minus_2
        fib_n_minus_2 = fib_n_minus_1
        fib_n_minus_1 = fib_n

print(f"The {n}th Fibonacci number is {fib_n}")

In [ ]:
## However, when used in part of a larger program, such a section can quickly become hard to understand
## Therefore it is often preferable to segment out parts into a function,

def fibonacci(n):
    if n == 0:
        return 0
    if n == 1:
        return 1
    fib_n_minus_2 = 0
    fib_n_minus_1 = 1
    for i in range(2, n + 1):
        fib_n = fib_n_minus_1 + fib_n_minus_2
        fib_n_minus_2 = fib_n_minus_1
        fib_n_minus_1 = fib_n
    return fib_n

def print_fibonacci(n, fib_n):
    print(f"The {n}th Fibonacci number is {fib_n}")

n = 10
fib_n = fibonacci(n)
print_fibonacci(n, fib_n)

## This allows the function `fibonacci` and `print_fibonacci` to be defined elsewhere, and reduced the main
## Code block that needs to be understood down to 3 lines

### Document your Functions

Using functions for readability of course only works if the purpose of a function is easily understood. While the names of our `fibonacci` and `print_fibonacci` functions are quite self-explanatory, this is in no way guaranteed for more complex functions. It is therefore strongly advised to get into the habit of documenting the purpose and use of one's self-defined functions. 

Ideally, this is done using a docstring, indicated by triple quotes (`"""`) at the start and end. The docstring provides a brief description of the function's purpose, its input arguments, and the output. This helps improve the readability and maintainability of the code, as it makes it easier for other developers to understand the function's purpose and usage.

In [ ]:
## Instead of this

def fibonacci(n):
    if n == 0:
        return 0
    if n == 1:
        return 1
    fib_n_minus_2 = 0
    fib_n_minus_1 = 1
    for i in range(2, n + 1):
        fib_n = fib_n_minus_1 + fib_n_minus_2
        fib_n_minus_2 = fib_n_minus_1
        fib_n_minus_1 = fib_n
    return fib_n

def print_fibonacci(n, fib_n):
    print(f"The {n}th Fibonacci number is {fib_n}")

## Do THIS:

def fibonacci(n):
    """
    Calculate the nth Fibonacci number using an iterative approach.

    Args:
        n (int): The position of the desired Fibonacci number in the sequence.

    Returns:
        int: The nth Fibonacci number.
    """
    if n == 0:
        return 0
    if n == 1:
        return 1
    fib_n_minus_2 = 0
    fib_n_minus_1 = 1
    for i in range(2, n + 1):
        fib_n = fib_n_minus_1 + fib_n_minus_2
        fib_n_minus_2 = fib_n_minus_1
        fib_n_minus_1 = fib_n
    return fib_n

def print_fibonacci(n, fib_n):
    """
    Print the nth Fibonacci number in a formatted message.

    Args:
        n (int): The position of the Fibonacci number in the sequence.
        fib_n (int): The nth Fibonacci number.
    """
    print(f"The {n}th Fibonacci number is {fib_n}")

In [ ]:
# as we have seen already, jupyter notebooks allow you to display these doc strings
fibonacci?

In [ ]:
# you can even get more information
fibonacci??

There are automatic tools that can generate boilerplate docstrings for you, where you can just enter your information.

### Avoid global variables

Global variables are a common source of bugs.
- Use functions with appropriate parameters, instead.
- If the number of parameters grows too large, it might can be a hint that a class would be a good idea.
- If you need to "preserve a state", i.e., store something between runs of a function, classes/objects can be a solution, too.

In [ ]:
# try to avoid this kind of definition in the global scope
your_global_variable = 42

def my_function(variable):
    """
    A function that prints the value of a variable.

    Args:
        variable: The variable to print.
    """
    # do not use global variables in functions,
    # but if you have to, make sure to document it
    print(f"Passed variable: {variable}\nGlobal Variable:{your_global_variable}")
my_function(1)

### Use existing Packages

Reinventing the wheel is not only time-consuming, but it can also make it difficult to follow your train of thought. Therefore it is often advisable and good style to make use of predefined and validate modules wherever possible. 

In [ ]:
## For example, we could implement the square root calculation ourself, using the Newton-Raphson method:

def sqrt_newton_raphson(x, epsilon=1e-10, max_iterations=1000):
    """
    Calculate the square root of a number using the Newton-Raphson method.

    Args:
        x (float): The number to find the square root of.
        epsilon (float, optional): The desired precision. Defaults to 1e-10.
        max_iterations (int, optional): The maximum number of iterations. Defaults to 1000.

    Returns:
        float: The square root of x.
    """
    guess = x / 2.0
    iteration = 0

    while abs(guess * guess - x) > epsilon and iteration < max_iterations:
        guess = 0.5 * (guess + x / guess)
        iteration += 1

    return guess

number = 25
sqrt_number = sqrt_newton_raphson(number)
print(f"The square root of {number} is approximately {sqrt_number}")


## Or, we could simply use the one provided in the math package:
import math

number = 25
sqrt_number = math.sqrt(number)
print(f"The square root of {number} is approximately {sqrt_number}")


### Define Your Own Modules

Similar to how segmenting code out into functions can improve readability, segmenting out groups of similar functions (or of functions with similar purposes) into their own modules can greatly improve how easy it is to follow your code. 

Additionally, this segmented approach makes it significantly easier to collaborate with others, as multiple people can contribute to a project without having to work on the same *file*.

For a refresher on how to define your own modules, you can go back and check the second half of lesson 06.

## Consistent Style 
The idea of a consistent style is to make the code look like it was written by a single person, even if it was written by many. This makes it easier to read and understand the code. To make this possible, there are style guides that define how code should be written.

### Use a Consistent Naming Scheme
While there are no hard rules for naming one's functions, variables, objects, or modules in Python, it is still advisable to have consistent naming schemes. Of course, these schemes can vary and should be quickly talked about with other collaborators. 

One common example scheme is this:
- Functions, Variables, Packages, … : `lower_case_with_underscores`
- Classes or Exceptions: `UpperCaseWithoutSpaces`
- Global constants (≠ global variables): `ALL_CAPS_WITH_UNDERSCORES`
- Avoid:
    - Single letters (Exception iterators, e.g.: i)
    - Names beginning with __ (Double _)
    - Unclear/confusing names

### PEP8 style guide
The most common style guide is certainly [PEP 8](https://peps.python.org/pep-0008/). There are more (and more stringent) style guides that build upon this.

### Automatic style checkers
There are a number of tools to automatically check or even apply/correct the coding styles using the command line or within editors or integrated development environments (IDEs):
- [Pylint](https://pylint.readthedocs.io/en/latest/)
- [Flake8](https://flake8.pycqa.org/en/latest/)
- [pycodestyle](https://pycodestyle.pycqa.org/en/latest/index.html)
- [Black](https://black.readthedocs.io/en/stable/index.html)

Some of these tools even work in jupyter notebooks. Let's try pycodestyle which we activate with these magic commands

In [ ]:
%load_ext pycodestyle_magic

In [ ]:
%pycodestyle_on

Check what is reported for the following cells (which are valid python code) and try to fix them according to what pycodestyle would recommend:

In [ ]:
a=1

In [ ]:
if 3 > 5:
  print(3)

In [ ]:
print("hi")

In [ ]:
x             = 1
y             = 2
long_variable = 3

In [ ]:
def function(arg_with_default = 3):
    return arg_with_default

You can also turn these checks off again

In [ ]:
%pycodestyle_off

In [ ]:
# should not complain now
a=1

### IDEs

Integrated development environments (IDE) are designed to maximize programmer productivity by providing a couple of utility features. 
An IDE normally consists of at least a source-code editor, build automation tools and a debugger. 
But we will more look into the following features that make your life easier. 

Modern IDEs provide you with the following features:
- **Syntax highlighting**: This feature show both the structures, the language keywords and the syntax errors with visually distinct colors and font effects. Helping you to spot errors quickly.
- **Code completion**: Modern IDEs have intelligent code completion, helping you to fix common mistakes and suggesting lines of code, for example a list of attributes and functions. This usually happens through popups while typing parameters of functions, and query hints related to syntax errors. 
- **Refactoring**: IDEs provide support for automated refactoring and tools to refactor your code in all place where they are used, e.g: change a variable or function name in all files where it is used. 
- **Version control**: Integrated version control help to interact with source repositories.
- **Debugging**: IDEs have integrated debugger, with support for setting breakpoints in the editor, visual rendering of steps, etc.
- **Code search and templates**: Searching for class and function declarations, usages, variable and field read/write and documentation is made easy. Also a lot of templates are available to help you write code faster.
- **Support for alternative languages**: Most IDEs are universal and support other languages by using plugins.
- **Many more**: Like online workspace, code snippets, code folding, etc.

Depending on your personal preferences and the project you are working on, you might want to use a different IDE. Here are some recommendations:
- [PyCharm](https://www.jetbrains.com/pycharm/) is a very powerful IDE that is free for students.
- [VS Code](https://code.visualstudio.com/) is the most used editor by Microsoft that has plenty of plugins to customize.
- [Spyder](https://www.spyder-ide.org/) is a scientific IDE that comes with Anaconda (you will most likely get this IDE while doing the F-Praktikum).

For now it is okay to start with a normal text editor to code your scripts. But as soon as you start to work on larger projects, you should consider using an IDE.


### Type hints
Python is a dynamically typed language, which means that you do not have to specify the type of a variable when you declare it. 
This can be very convenient, but it can also lead to bugs that are hard to find.
Type hints are introduced in Python 3.5 and are a way to specify the type of a variable, function, or class. 
These hints are not enforced by the Python interpreter, but they can be checked by external tools and also IDEs utilize them to provide better code completion and error checking. 
Therefore, it is a good idea to use type hints in your code.

Type hints are specified using the `:` operator in the head of a function definition, after the variable name.

In [ ]:
def add(
    a: int, # developer tells you a and b should be integers
    b: int,
) -> int: # developer tells you that the return value is also an integer
    return a + b

print(add(1,2))
print(add(1.,2.))

As you can see type hints are not enforced by the Python interpreter, but they can be checked by external tools.
You as developer need to take care that the types are correct.

There are several tools that can check your code for type hints, e.g. [mypy](http://mypy-lang.org/).
IDEs like PyCharm can also use type hints to provide better code completion and error checking.

Type hints:
- basic types: `int`, `float`, `str`, `bool`, `None`
- collection types: `list`, `tuple`, `dict`, `set` with elements in square brackets, e.g. `list[int]` or `tuple[str, float]`
- class instances are type hinted with the class name, e.g. for numpy arrays `numpy.ndarray`
- There is also the typing module that provides more complex types, e.g. `Any`, saying anything can go inside or `Union` for multiple types.

# Further reading / self study

- You can make a script callable just by its name (without writing `python3 <name.py>`) using a [shebang line](https://realpython.com/python-shebang/)
- In larger projects you should write software tests that ensure that the separate parts of your code do what they are supposed to do. In this way you can also avoid many errors that would be introduced by changes to the code. A good starting point is the [Unit testing framework `unittest`](https://docs.python.org/3/library/unittest.html)
- Your projects have dependencies and these need to be managed, on the other hand you do not want to mix these dependencies with the dependencies of other projects, e.g. if both projects use the same package but different versions of it. This is where `virtualenvs` comes into play. They create an isolated Python environment. There are many ways to handle these a comprehensive overview is given [here](https://realpython.com/python-virtual-environments-a-primer/).
  
## Installing python on your local computer
- Always make sure to use a Python 3 version, not Python 2.
- There are general [instructions](https://wiki.python.org/moin/BeginnersGuide/Download) for bare python installations for many operating systems.
- For Linux, your package management system will provide a Python installation.
- For Mac, there are different ways, including `brew`.
- For Windows, we recommend installing the [Anaconda distribution](https://docs.anaconda.com/free/anaconda/install/windows/) that comes with many packages preinstalled.

## End of part 1

This is the end of the part you should read at home. Everything below this cell will be topic in the next exercise session and you don't need to look at this now.

## Interactive Part
The double pendulum is a simple physical system that exhibits chaotic behavior. 
The result is a quiet complicated motion of the pendulum, which is hard to predict.

The reason for this is a strong sensitivity to initial conditions.
The equations of motion depend on following parameters: 
- the initial angles $\theta_{1}$ and $\theta_{2}$
- the initial angular velocities $\dot{\omega_{1}}$ and $\dot{\omega_{2}}$
- the lengths of the pendulums $l_{1}$ and $l_{2}$
- the masses of the pendulums $m_{1}$ and $m_{2}$
- the gravitational acceleration $g$


<div>
<img src="https://web.mit.edu/jorloff/www/chaosTalk/double-pendulum/dbl_pendulum.gif" width="200"/>
</div>

Fold and skip the optional part if you do not need an introduction about the double pendulum.


**Optional theoretical background:**  
The equations of motion describe the evolution about how an object will move, according to its initial conditions.
These equations can be derived by utilizing the Lagrangian and the usage of the Euler-Lagrangian-Formalism.

Oversimplified, a Lagrangian is a function that describes the dynamics of a system by terms of energy, and not by its force. 
The Euler-Lagrange Formalism is used to get the equation of motion from the Lagrangian. 

If you heard Theoretical Physics I this term should be familiar. 
But this is not the topic of this exercise, therefore we will just use the equations of motion and do not care about their derivation.

The coordinates of the system:  

$$x_1 = L_1 \cdot \sin(\theta_1)$$
$$y_1 = -L_2 \cdot \cos(\theta_1)$$

$$x_2 = L_1 \sin(\theta_1) + L_2 \sin(\theta_2) = x_1 + L2 \sin(\theta_1)$$
$$y_2 = -L_1 \cos(\theta_1) - L_2\cos(\theta_2) = y_1 - L2 \cos(\theta_2)$$

$(x_1, y_2)$ describing the position of the mass $m_1$ and $(x_2, y_2)$ the position of mass $m_2$.

Solving the Euler-Lagrange equation results in:
$$
    \ddot{\theta_{1}} = \frac{
        -g \cdot (2m_{1} + m_{2}) \cdot \sin(\theta_{1}) 
        - m_{2} \cdot g \cdot \sin(\theta_{1} - 2\theta_{2})
        - 2\sin(\theta_{1} - \theta_{2}) m_{2} \cdot (\dot{\theta_{2}}^{2} \cdot L_{2} 
        + \dot{\theta_{1}}^{2} \cdot L_{1} \cdot \cos(\theta_{1} - \theta_{2}))
                            }
                            {
        L_{1} \cdot (2m_{1} + m_{2} - m_{2} \cdot \cos(2\theta_{1} - 2\theta_{2}))
                        }
$$


$$
    \ddot{\theta_{2}} = \frac{
        2\sin(\theta_{1} - \theta_{2}) \cdot (\dot{\theta_{1}}^{2} \cdot L_{1} \cdot (m_{1} + m_{2}) 
        + g \cdot (m_{1} + m_{2}) \cdot \cos(\theta_{1})
        + \dot{\theta_{2}}^{2} \cdot L_{2} \cdot m_{2} \cdot \cos(\theta_{1} - \theta_{2}))
                        }
                        {
        L_{2} \cdot (2m_{1} + m_{2} - m_{2} \cdot \cos(2\theta_{1} - 2\theta_{2}))}
$$

$\theta$ is our angle, $\dot{\theta}$ is the angular velocity and $\ddot{\theta}$ is the angular acceleration.
Solving this kind of equation is predominantly done numerically, which is why we will use the `scipy` package for this task.

To do so we need to rewrite this equation as first order differential equation by rewriting the change of angle as angular velocity and the change of angular velocity as angular acceleration: $\dot{\theta_{1}} = \omega_{1}$, $\dot{\theta_{2}} = \omega_{2}$.
Inserting this and doing some moving around of expressions this leads to the following equations for the second order derivatives:

$$ 
    \dot{\omega_{1}} = \frac{
        m_{2} \cdot L_{1} \cdot \omega_{1}^{2} \cdot \sin(\theta_{2} - \theta_{1}) \cdot \cos(\theta_{2} - \theta_{1})
        + m_{2} \cdot g \cdot \sin(\theta_{2}) \cdot \cos(\theta_{2} - \theta_{1})
        + m_{2} \cdot L_{2} \cdot \omega_{2}^{2} \cdot \sin(\theta_{2} - \theta_{1})
        - (m_{1} + m_{2}) \cdot g \cdot \sin(\theta_{1})
}
{
        (m_{1} + m_{2}) \cdot L_{1} - m_{2} \cdot L_{1} \cdot \cos(\theta_{2} - \theta_{1})^{2}
}
$$

$$
    \dot{\omega_{2}} = \frac{
        - m_{2} \cdot L_{2} \cdot \omega_{2}^{2} \cdot \sin(\theta_{2} - \theta_{1}) \cdot \cos(\theta_{2} - \theta_{1})
        + (m_{1} + m_{2}) \cdot g \cdot \sin(\theta_{1}) \cdot \cos(\theta_{2} - \theta_{1})
        - (m_{1} + m_{2}) \cdot L_{1} \cdot \omega_{1}^{2} \cdot \sin(\theta_{2} - \theta_{1})
        - (m_{1} + m_{2}) \cdot g \cdot \sin(\theta_{2})
}
{
        (m_{1} + m_{2}) \cdot L_{2} - m_{2} \cdot L_{2} \cdot \cos(\theta_{2} - \theta_{1})^{2}                    
}
$$

With these 4-equations we can use a numerical differential equation solver to solve the equations of motion.


**Task**: In the following example you get a code snippet from your colleague, where he implemented the double pendulum. 
Your task is to refactor the code to make it more readable and maintainable. 
Before doing so try to find out what the code does by playing around with it. 

Remember when writing your program you try to abstract the implementation details from the user. 
A typical user does not need to know the internal implementation of the double pendulum. 
The user only needs to know how to use the double pendulum, what the parameters are and what the output is.

**Note:** Sadly Jupyter doesn't support animations by default and packages that could fix this issue are not installed here.
As a workaround we create an animated `gif` and play this in the notebook.
For some reason, creating the gif takes a lot of time (more than the actual simulation).
Please leave the time you want to simulate at 10 s.  
If you are in a pool room, you can copy pase the code to a python script.
Remove `from IPython.display import Image` and replace:
```python
writer = animation.PillowWriter(fps=20)
ani.save("animation.gif", writer)

# display the animation
Image(filename="animation.gif")
```
by
```python
plt.show()
```
Then you can run the code in the terminal on your computer and you don't need to wait for the animation.
In that case you can also look at higher simulation times.

**Task a:** Read the code and try to understand it.
Don't hesitate to ask your tutor in case things remain unclear.
It might help you to add comments, it's up to you.

**Task b:** How can you improve the code?
Why is this code not good?
Compare with the concepts you learned above.
There are already a lot of hints in the code that will guide you.
Finally, make a plan what you want to change and how you want to do it.
Discuss this plan with your tutor.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
from numpy import cos, sin
import scipy.integrate
from IPython.display import Image

# question: why is a config dictionary a good idea, instead of just using global variables?
# tip: think about the asterisk unpacking operator (*, **)
config = {
    "G": 9.8,  # acceleration due to gravity, in m/s^2
    "L1": 1.0,  # length of pendulum 1 in m
    "L2": 3.0,  # length of pendulum 2 in m
    "M1": 2.0,  # mass of pendulum 1 in kg
    "M2": 1.0,  # mass of pendulum 2 in kg
    "t_stop": 10.0,  # how many seconds to simulate
}

config["L"] = config["L1"] + config["L2"]  # maximal length of the combined pendulum

# question: how could type hint help us understand this function?

def eq(t, state):
    # question: is this a good docstring?
    """
    derive second order ODE from first order ODE

    """
    # question: is this a good name?

    # tip: try to bring comments into play that answers the question:
    # - why is this variable here? Add context, the purpose and goal.
    # - use simple language (no jargon) and short sentences.
    # - include reference link if necessary (for example for a formula, or database)
    # - write comments while writing the code (when its still fresh)

    dydx = np.zeros_like(state)

    # question: global variables are bad practice, how can we improve this?
    M1, M2, L1, L2, G = config["M1"], config["M2"], config["L1"], config["L2"], config["G"]

    theta1, omega1, theta2, omega2 = state

    dydx[0] = omega1

    delta = theta2 - theta1

    # question: is this equation correct? Make it more readable
    # tip: use more descriptive variable names (e.g. numerator, denominator, omega_1_velocity, ...)
    # tip: when facing large equations, it is often helpful to split them up into smaller parts
    # e.g. split up the equation into the numerator and denominator
    # e.g. split up by + - operations

    dydx[1] = ((M2 * L1 * omega1**2 * sin(delta) * cos(delta) + M2 * G * sin(theta2) * cos(delta) + M2 * L2 * omega2**2 * sin(delta) - (M1+M2) * G * sin(theta1))) / ((M1+M2) * L1 - M2 * L1 * cos(delta) * cos(delta))

    dydx[2] = omega2

    # question: isn't most of the denominator the same as from equation 1?
    dydx[3] = ((- M2 * L2 * omega2**2 * sin(delta) * cos(delta) + (M1+M2) * G * sin(theta1) * cos(delta) - (M1+M2) * L1 * omega1**2 * sin(delta) - (M1+M2) * G * sin(theta2))) / ((M1+M2) * L2 - M2 * L2 * cos(delta) * cos(delta))
    return dydx


# question: can you summarize the next lines of code in one sentence?
# set up interval for integration
dt = 0.02
t = np.arange(0, config["t_stop"], dt)

# initial state
# th1 and th2 are the initial angles (degrees)
# w10 and w20 are the initial angular velocities (degrees per second)
th1 = 0.0
w1 = 0.0
th2 = 60.0
w2 = 0.0

state = np.radians([th1, w1, th2, w2])

# integrate the ODE using Euler's method
y = np.empty((len(t), 4))
y[0] = state

# a less accurate method would be to use Euler's method
# for i in range(1, len(t)):
#     y[i] = y[i - 1] + eq(t[i - 1], y[i - 1]) * dt

# Runge-Kutta methods are more accurate and are used in scipy.integrate.solve_ivp
# question: t[[0, -1]] is quiet hard to understand?
y = scipy.integrate.solve_ivp(eq, t[[0, -1]], state, t_eval=t).y.T

# question: maybe its a good idea to pack this into a function?
L1, L2, L = config["L1"], config["L2"], config["L"]
x1 = L1 * sin(y[:, 0])
y1 = -L1 * cos(y[:, 0])

x2 = L2 * sin(y[:, 2]) + x1
y2 = -L2 * cos(y[:, 2]) + y1

# question: what is the purpose of this code block?
fig = plt.figure(figsize=(5, 4))
ax = fig.add_subplot(autoscale_on=False, xlim=(-L, L), ylim=(-L, 1.0))
ax.set_aspect('equal')
ax.grid()

line, = ax.plot([], [], 'o-', lw=2)
trace, = ax.plot([], [], '.-', lw=1, ms=2)
time_template = 'time = %.1fs'
time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)


def animate(i):
    # question: what is i, what is this function do, shouldn't we pass variables?
    thisx = [0, x1[i], x2[i]]
    thisy = [0, y1[i], y2[i]]

    history_x = x2[:i]
    history_y = y2[:i]

    line.set_data(thisx, thisy)
    trace.set_data(history_x, history_y)
    time_text.set_text(time_template % (i * dt))
    return line, trace, time_text


# question: why the * 1000, try to avoid magic numbers?
# question: look this function up, how to pass arguments to it? See text above for link
ani = animation.FuncAnimation(
    fig,
    animate,
    len(y),
    interval=dt * 1000,
    blit=True
)

writer = animation.PillowWriter(fps=20)
ani.save("animation.gif", writer)

# display the animation
Image(filename="animation.gif")

# end question: which part of the code would go into a main function?

### Some useful hints:

#### Input parameter handling

We collected all the parameters for you that were global variables in the original code and structured them in a useful way:
```python
CONSTANTS = {
    "L1":1, # length of pendulum 1 in m
    "L2":3, # length of pendulum 2 in m
    "m1":2, # mass of pendulum 1 in kg
    "m2":1, # mass of pendulum 2 in kg
    "g":9.81, # acceleration due to gravity in m/s^2
}

INITIAL_CONDITIONS = {
    "theta1" : 0.0, # degree
    "angular_velocity1" : 0.0, # degree per second
    "theta2" : 60.0, # degree
    "angular_velocity2" : 0.0, # degree per second
}

SIMULATION_SETTINGS = {
    "duration": 10, # duration of simulation in seconds
    "dt": 0.02, # time step in seconds, the finer the more precise
    "fps": 20, # frames per second for the animation
}
```
Your "main" method should take these dictionaries as input.

**Side Note:** To have this as a standalone python script, we would need to handle the arguments somehow.
In projects with a low number of parameters, one would use an argparser and catch every single parameter from the terminal.
For programs with many parameters this can be very nasty.
An alternative is to create a config file containing all the parameters.
In that case, the only parameter of the script is the path to the config file and then the actual parameters are read from the file.
Given that we didn't cover config files in this notebook, we just put the parameters in a separate python script or separate jupyter notebook cell and import project as a module and call the "main" function from there.

#### Avoiding global variables when using functions that call other functions

There are two functions in the code that each take another function as parameter and call it internally.
One of them is the `scipy.integrate.solve_ivp` function that calls internally the `derivs` function.
The `scipy.integrate.solve_ivp` automatically passes the arguments for `t` and `state` to the `derivs` function.
The question is now: How are further parameters of the `derivs` function handled?
To avoid using global variables you might want to add further parameters to this function.
The answer can be found in the [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html).
The `scipy.integrate.solve_ivp` function has a parameter `args` which can be a tuple of arguments that is passed to the `derivs` function.

The other function is matplotlib's `animation.FuncAnimation` function which takes the `animate` function as argument.
Here is the [documentation](https://matplotlib.org/stable/api/_as_gen/matplotlib.animation.FuncAnimation.html) of this function.
How do you pass further arguments to the `animate` function?
It is very similar!

**Task c:** Do what you discussed so far and refactor the code.

In [ ]:
# BEGIN-LIVE
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint, solve_ivp
import matplotlib.animation as animation
import matplotlib
from IPython.display import Image


# Show this optimized solution as "good example" in the tutorial at the end

# Defining the double pendulum equations of motion
def double_pendulum_equations(t: list[float], state: np.ndarray, pendulum_constants) -> np.ndarray:
    """
    Defines the system of differential equations for a double pendulum.

    Parameters:
    t (list[float]): Time as a numpy array.
    state (np.ndarray): Initial values. These values are the angles and angular velocities: [theta1, theta1_dot, theta2, theta2_dot].
    pendulum_constants (dict): A dictionary containing the physical constants of the pendulum system.
    It should include the lengths (L1, L2), masses (m1, m2), and gravitational acceleration (g).

    Returns:
    np.ndarray: A list of differential equations representing the system's motion.

    The function first unpacks the state variables and pendulum constants. It then defines four helper functions to calculate
    the accelerations and velocities of the two pendulums. These helper functions are used to construct the system of
    differential equations, which is returned as a numpy array.
    """
    theta1, omega1, theta2, omega2 = state

    VALUES = pendulum_constants.copy()
    VALUES["omega1"] = omega1
    VALUES["omega2"] = omega2
    VALUES["theta1"] = theta1
    VALUES["theta2"] = theta2

    def omega1_velocity(L1: float, L2: float, m1: float, m2: float, g: float, theta1: float, theta2: float, omega1: float, omega2: float, **kwargs):
        # split (by + and -) equation into smaller parts for readability
        delta = theta2 - theta1

        nominator_1 = (
            m2 * L1 * omega1**2 * np.sin(delta) * np.cos(delta)
            + m2 * g * np.sin(theta2) * np.cos(delta)
            + m2 * L2 * omega2**2 * np.sin(delta)
            - (m1+m2) * g * np.sin(theta1)
        )
        denominator_1 = (
            (m1+m2) * L1
            - m2 * L1 * np.cos(delta)**2
        )

        return nominator_1/denominator_1

    def omega2_velocity(L1: float, L2: float, m1: float, m2: float, g: float, theta1: float, theta2: float, omega1: float, omega2: float, **kwargs):
        delta = theta2 - theta1

        nominator_2 = (
            (- m2 * L2 * omega2**2 * np.sin(delta) * np.cos(delta)
            + (m1 + m2) * g * np.sin(theta1) * np.cos(delta)
            - (m1 + m2) * L1 * omega1**2 * np.sin(delta)
            - (m1 + m2) * g * np.sin(theta2))
        )
        denominator_2 = (L2 / L1) * (
            (m1 + m2) * L1
            - m2 * L1 * np.cos(delta)**2
        )
        return nominator_2 / denominator_2

    # prepare numpy array to store the differential equations results
    differential_equations = np.zeros_like(state)

    differential_equations[0] = omega1
    differential_equations[1] = omega1_velocity(**VALUES)
    differential_equations[2] = omega2
    differential_equations[3] = omega2_velocity(**VALUES)

    return differential_equations


def animate(
    index: int,
    dt: float,
    arrays: list[np.ndarray],
    matplotlib_elements: tuple[matplotlib.lines.Line2D, matplotlib.lines.Line2D, matplotlib.text.Text]
) -> tuple[matplotlib.lines.Line2D, matplotlib.lines.Line2D, matplotlib.text.Text]:
    """
    Updates the positions of the pendulums for each frame in the animation.

    Parameters:
    index (int): The current frame index.
    dt (float): The time step between frames.
    arrays (list[np.ndarray]): A list of numpy arrays representing the x and y coordinates of the pendulums.
    matplotlib_elements (tuple): A tuple containing the line, trace, and time text elements to be updated.

    Returns:
    tuple: The updated line, trace, and time text elements.
    """
    x1, y1, x2, y2 = arrays
    line, trace, time_text = matplotlib_elements

    thisx = [0, x1[index], x2[index]]
    thisy = [0, y1[index], y2[index]]

    history_x = x2[:index]
    history_y = y2[:index]

    line.set_data(thisx, thisy)
    trace.set_data(history_x, history_y)
    time_template = f'time = {index * dt:.1f}s'
    time_text.set_text(time_template)
    return line, trace, time_text

def get_coordinates(L1: float, L2: float, theta1: float, theta2: float) -> tuple[float, float, float, float]:
    x1 = L1 * np.sin(theta1)
    y1 = -L1 * np.cos(theta1)
    x2 = x1 + L2 * np.sin(theta2)
    y2 = y1 - L2 * np.cos(theta2)
    return (x1, y1), (x2, y2)

def save_animation(animation_inst: matplotlib.animation, dst_path: str, fps: float = 30) -> None:
    writer = animation.PillowWriter(fps=fps)
    animation_inst.save(dst_path, writer)

def main(constants, initial_conditions, simulation_settings):
    # initial state
    # translate the initial conditions to radians, since trig function use them
    initial_state = np.radians(
        [
            initial_conditions["theta1"],
            initial_conditions["angular_velocity1"],
            initial_conditions["theta2"],
            initial_conditions["angular_velocity2"],
        ]
    )

    # integrate the differential equation
    # create a time array from 0..t_stop sampled at 0.02 second steps
    dt = np.array(simulation_settings["dt"])
    t_stop = np.array(simulation_settings["duration"])
    t = np.arange(0, t_stop, dt)

    # set initial state as 0-th time step
    states = np.empty((len(t), 4))
    states[0] = initial_state

    # solve equation system
    # the result is a list of states, each state containing the angles and angular velocities of the two pendulums
    # [angle1, angular_velocity1, angle2, angular_velocity2]
    states = solve_ivp(
        fun=double_pendulum_equations,
        t_span=(0, t_stop),
        y0=initial_state,
        method="RK45",  # runge-kutta method
        args=(constants,),
        t_eval=t
    ).y.T

    #     non scipy using Euler's method
    #     for i in range(1, len(t)):
    #         states[i] = states[i - 1] + double_pendulum_equations(t[i - 1], states[i - 1], constants) * dt
    # plotting task
    COORDINATE_VALUES = {
        "L1": constants["L1"],
        "L2": constants["L2"],
        "theta1": states[:,0],
        "theta2": states[:,2],
    }

    # get coordinates in cartesian space
    (x1, y1), (x2, y2) = get_coordinates(**COORDINATE_VALUES)

    # set up the figure, the axis, and the plot element we want to animate
    additional_spacer = 0.1
    L = constants["L1"] + constants["L2"] + additional_spacer

    fig = plt.figure(figsize=(5, 4))
    ax = fig.add_subplot(autoscale_on=False, xlim=(-L, L), ylim=(-L, 1.0))
    ax.set_aspect("equal")
    ax.grid()

    # define lines and trace of the pendulum and time text
    line, = ax.plot([], [], "o-", lw=2)
    trace, = ax.plot([], [], ".-", lw=1, ms=2)
    time_text = ax.text(0.05, 0.9, "", transform=ax.transAxes)

    # animate the plot
    ani = animation.FuncAnimation(
        fig,
        animate,
        len(states),
        fargs=(dt, [x1, y1, x2, y2], [line, trace, time_text]),
        interval=dt * 1000,  # in milliseconds
        blit=True,
    )

    animation_saving_object = save_animation(
        animation_inst=ani,
        dst_path="animation_2.gif",
        fps=simulation_settings["fps"],  # frames per second, generating the gif is not computationally expensive
    )

# API to start the simulation
CONSTANTS = {
    "L1": 1,  # length of pendulum 1 in m
    "L2": 3,  # length of pendulum 2 in m
    "m1": 2,  # mass of pendulum 1 in kg
    "m2": 1,  # mass of pendulum 2 in kg
    "g": 9.81,  # acceleration due to gravity in m/s^2
}

INITIAL_CONDITIONS = {
    "theta1": 0.0,  # degree
    "angular_velocity1": 0.0,  # degree per second
    "theta2": 60.0,  # degree
    "angular_velocity2": 0.0,  # degree per second
}

SIMULATION_SETTINGS = {
    "duration": 10,  # duration of simulation in seconds
    "dt": 0.02,  # time step in seconds, the finer the more precise
    "fps": 20,  # frames per second for the animation
}


main(CONSTANTS, INITIAL_CONDITIONS, SIMULATION_SETTINGS)

# display the animation
Image(filename="animation_2.gif")

# END-LIVE

**Task d:** Compare the the results of your refactored version with the ones from the original code. Is it the same?

#### Some critical comments

This is a very sensitive and fragile system.
In general you shouldn't take the simulation too serious (especially for long time periods).
It for sure gives you an impression how this pendulum behaves but you shouldn't expect that you can predict the position of the pendulum and the path it took to get there 1 minute after the start.
Imagine you have a pendulum (fixed lengths and fixed masses) the amount of different movements (different paths) this pendulum can do within one minute seems to be infinite.
But there are only two parameters describing the initial state (the two angles).
You can imagine that a tiny change in one of the angles can cause a big difference.
In the first seconds it might result in a very small deviation, this small deviation might cause a slightly larger deviation in the next seconds and so on.
After one minute the state of the pendulum is completely different to the unchanged version.
The same effect you can also observe when you have some small inaccuracies in your simulation.
For numerical simulations you can always expect that it is not perfectly accurate.

A nice example where you can actually see the inaccuracy of the simulation is when you set both angles to 180 degrees.
In real live it is impossible to get both angles perfectly but in the simulation you can do it.
Now the pendulum should stay where it is but also in the simulation it starts moving after some time.

Due to those inaccuracies you really can't predict the behaviour of the pendulum over a long time.
For long times this can also cause a different behaviour of the pendulum using the original and your refactored code.